In [1]:
import numpy as np
import pandas as pd
import shutil
from shutil import copyfile
from tqdm import tqdm
import os

pd.set_option('display.max_rows', 1000+1)


#### 1, Read in GODESS glycans from all sources, put them in a new directory

In [2]:
source_list = ['Experimental_1D/', 'Theoretical_NewGlycans_MaxLength20/', 'D2O/', 'Glycoscience_Linear/', 
              'Glycoscience_Linear_second_part/', 'Glycoscience_Linear_Nonlinear/']
out_dir = 'dataset/Godess_carbon/'

prev_source_list = []
prev_file_name = []

new_file_name = []

In [4]:
i = 1
ct = 0
for s in source_list:
    s_files_names = os.listdir(s)
    ct += len(s_files_names)
    for f in s_files_names:
        s_files_path = os.path.join(s, f)
        
        temp_glycan_all_files = os.listdir(s_files_path)
        
        if 'PDB.pdb' in temp_glycan_all_files:
            pass
        else:
            print(s_files_path, 'is missing PDB file')
        
        if 'c_tsv_stat.txt' in temp_glycan_all_files:
            pass
        else:
            print(s_files_path, 'is missing c_tsv_stat.txt')
        
        if 'h_tsv_stat.txt' in temp_glycan_all_files:
            pass
        else:
            print(s_files_path, 'is missing h_tsv_stat.txt')
        
        
        if ('PDB.pdb' in temp_glycan_all_files) and ('c_tsv_stat.txt' in temp_glycan_all_files) and \
            ('h_tsv_stat.txt' in temp_glycan_all_files):
            pass
            
#             new_dir_name = os.path.join(out_dir, str(i))
            
#             os.makedirs(new_dir_name)
            
#             old_pdb_path = os.path.join(s_files_path, 'PDB.pdb')
            
#             old_c_label_path = os.path.join(s_files_path, 'c_tsv_stat.txt')
            
#             old_h_label_path = os.path.join(s_files_path, 'h_tsv_stat.txt')
            
            
#             new_pdb_path = os.path.join(new_dir_name, 'PDB.pdb')
            
#             new_c_label_path = os.path.join(new_dir_name, 'c_tsv_stat.txt')
            
#             new_h_label_path = os.path.join(new_dir_name, 'h_tsv_stat.txt')
            
            
#             copyfile(old_pdb_path, new_pdb_path)
            
#             copyfile(old_c_label_path, new_c_label_path)
            
#             copyfile(old_h_label_path, new_h_label_path)
            
#             prev_source_list.append(s)
#             prev_file_name.append(f)
#             new_file_name.append(i)
            
            
#             i += 1

Experimental_1D/194 is missing c_tsv_stat.txt
Experimental_1D/194 is missing h_tsv_stat.txt
Experimental_1D/110 is missing c_tsv_stat.txt
Experimental_1D/110 is missing h_tsv_stat.txt
Experimental_1D/78 is missing c_tsv_stat.txt
Experimental_1D/78 is missing h_tsv_stat.txt
Experimental_1D/145 is missing c_tsv_stat.txt
Experimental_1D/145 is missing h_tsv_stat.txt
Experimental_1D/206 is missing c_tsv_stat.txt
Experimental_1D/206 is missing h_tsv_stat.txt
Experimental_1D/20 is missing c_tsv_stat.txt
Experimental_1D/20 is missing h_tsv_stat.txt
Experimental_1D/100 is missing c_tsv_stat.txt
Experimental_1D/100 is missing h_tsv_stat.txt
Experimental_1D/209 is missing c_tsv_stat.txt
Experimental_1D/209 is missing h_tsv_stat.txt
Experimental_1D/114 is missing c_tsv_stat.txt
Experimental_1D/114 is missing h_tsv_stat.txt
Experimental_1D/386 is missing c_tsv_stat.txt
Experimental_1D/386 is missing h_tsv_stat.txt
Experimental_1D/28 is missing c_tsv_stat.txt
Experimental_1D/28 is missing h_tsv_sta

In [5]:
ct

2629

In [4]:
out_dir

'dataset/Godess_carbon/'

In [5]:
pd.DataFrame([prev_source_list, prev_file_name, new_file_name]).T.to_csv('new_matching_table.csv', index = False)

In [7]:
pd.read_csv('new_matching_table.csv')

,0,1,2
0,Experimental_1D/,33a,1
1,Experimental_1D/,67x,2
2,Experimental_1D/,50x,3
3,Experimental_1D/,338,4
4,Experimental_1D/,103b,5
...,...,...,...
2411,Glycoscience_Linear_Nonlinear/,96,2412
2412,Glycoscience_Linear_Nonlinear/,77,2413
2413,Glycoscience_Linear_Nonlinear/,104,2414
2414,Glycoscience_Linear_Nonlinear/,105,2415


#### 2, Read in pdb dataset

In [4]:
data_dir = 'dataset/Godess_carbon/'
pdb_out_dir = 'dataset/Godess_carbon_gnn/Godess_carbon_unmatched_pdb/'
csv_out_dir = 'dataset/Godess_carbon_gnn/Godess_carbon_unmatched_csv/'
name_out_dir = 'dataset/Godess_carbon_gnn/Godess_carbon_name.csv'

name_list = []
folder_names = os.listdir(data_dir)

In [5]:
def experiment_process_single_pdb(f):
    temp_shift = []
    for f_l in f:
        if 'HETATM' in f_l:
            temp_shift.append(f_l)

    new_shift = []
    for i in range(len(temp_shift)):
        temp_l = temp_shift[i].split(' ')
        temp_l_s = [i for i in temp_l if i != '']
        new_shift.append(temp_l_s[0:9] + list(temp_l_s[-2]))
            
    return new_shift

def extract_glycan_name(f):
    
    glycan_name = 'Missing glycan name'
    
    for f_l in f:
        if 'COMPND' in f_l:
            glycan_name = f_l.split('= ')[-1].split(' \n')[0]
            
    return glycan_name

def extract_process_single_label(c):
    new_g = []
    for c_l in c:
        c_temp = c_l.split('\t')

        new_l = []
        for i in c_temp:
            if '\n' in i:
                i = i.split('\n')[0]
            new_l.append(i)

        new_g.append(new_l)
        
    df = pd.DataFrame(new_g)
    
    df = df.rename(columns=df.iloc[0])
    df = df.tail(-1)
    df.index = range(len(df))
    

    
    
    return df


In [6]:

# for i in range(len(folder_names)):
    
#     folder = folder_names[i]
#     pdb_path = os.path.join(data_dir, folder, 'PDB.pdb')
#     label_path = os.path.join(data_dir, folder, 'c_tsv_hyb.txt')
    
#     # pdb shifts
#     with open(pdb_path) as f:
#         f = f.readlines()
#         new_shift = experiment_process_single_pdb(f)
#         g_name = extract_glycan_name(f)
        
#         df_pdb = pd.DataFrame(new_shift)
        
        
#         # glycan names not included, in 549
#         if g_name == 'Missing glycan name':
#             continue
        
#         # some pdb contain redundant columns
#         if df_pdb.shape[1] > 10:
            
#             df_pdb = df_pdb.iloc[:,:-1]
            
#         df_pdb.columns = ['HETATM', 'Atom_num', 'Atom_name', 'Residual_name', 'Bound', 
#                           'Residual_num', 'x', 'y', 'z', 'Atom_type']
        
        
#         df_pdb_out_path = os.path.join(pdb_out_dir, str(i) + '.pdb.csv')
        
#         df_pdb.to_csv(df_pdb_out_path, index = False)
        
#         name_list.append(g_name)
        
#     # nmr labels
#     with open(label_path) as c:
#         c = c.readlines()
#         df_label = extract_process_single_label(c)
    
#         # some pdb shift contain more than 12 labels, this happens mostly from the 
#         if (df_label.shape[1] > 12):

#             df_label = df_label.iloc[1:]
#             df_label = df_label.dropna(axis=1, how='all')
    
#         # one pdb contain only C1-C5, this happens because of the type of the monosaccharide
#         elif (df_label.shape[1] < 9):
#             print(df_pdb_out_path)
#             df_label['C6'] = None

#         df_label_out_path = os.path.join(csv_out_dir, str(i) + '.csv')
        
#         df_label.to_csv(df_label_out_path, index = False)
        
#     df_name = pd.DataFrame([str(i), name_list]).T
#     df_name.to_csv(name_out_dir, index = False)

#### 3, Simple Allignment

In [7]:
pdb_dir = 'dataset/Godess_carbon_gnn/Godess_carbon_unmatched_pdb/'
csv_dir = 'dataset/Godess_carbon_gnn/Godess_carbon_unmatched_csv/'
out_labeled_pdb_dir = 'dataset/Godess_carbon_gnn/Godess_carbon_labeled_pdb/'

In [8]:
carbon_list = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
hydrogen_list_1_6 = ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9']

In [9]:
pdb_files = os.listdir(pdb_dir)
csv_files = os.listdir(csv_dir)

In [10]:
three_letter_non_residual_list = ['PDa', 'ACY']
incorrect_pdb_list = ['1983.pdb.csv', '690.pdb.csv', '441.pdb.csv', '737.pdb.csv']

In [22]:
# ct = 0
# for i in tqdm(range(len(pdb_files))):
# # for i in range(len(pdb_files)):
# # for i in range(1):
# # for i in [30]:
# # for i in [40]:
# # for i in [40]:

# # for i in range(1786, len(pdb_files)):
#     pdb_name = pdb_files[i]
#     csv_name = pdb_name.replace('.pdb.csv', '.csv')
    
#     pdb_name_path = os.path.join(pdb_dir, pdb_name)
#     csv_name_path = os.path.join(csv_dir, csv_name)
    
#     pdb_f = pd.read_csv(pdb_name_path)
#     csv_f = pd.read_csv(csv_name_path)
    
#     csv_f = csv_f.loc[csv_f['Residue'] != 'Ac']
#     csv_f.index = range(len(csv_f))
    
    
#     if pdb_name in incorrect_pdb_list:
#         continue
    
#     if len(csv_f) == len(np.unique(pdb_f['Residual_num'])):
#         pass
#     else:
# #         print(csv_name)
#         ct += 1
    
#     temp_labels = np.repeat(-1.0, len(pdb_f))
    
#     prev_residual_name = ''
    
#     # if we see and ACY, skip the label of it.
#     diff = 1
#     for j in range(len(pdb_f)):
        
#         current_atom_type = pdb_f.loc[j, ['Atom_name']].values[0]
#         current_residual = pdb_f.loc[j, ['Residual_num']].values[0]
        
#         current_residual_name = pdb_f.loc[j, ['Residual_name']].values[0]
        
#         if (current_residual_name in three_letter_non_residual_list) and \
#             (prev_residual_name not in three_letter_non_residual_list):
#             diff += 1
        
#         prev_residual_name = current_residual_name
            
#         if current_atom_type in carbon_list:
#             corresponding_residual = current_residual - diff
            
#             if csv_f.loc[corresponding_residual, [current_atom_type]].values[0] is not None:
                
#                 current_shift_val = csv_f.loc[corresponding_residual, [current_atom_type]].values[0]
                
#                 if current_shift_val == '?':
#                     current_shift_val = -1
                    
#                 # only consider the confident score
#                 if pd.isnull(csv_f.loc[corresponding_residual, ['Trust']].values[0]):
#                         current_shift_val = -1
                        
#                 elif int(csv_f.loc[corresponding_residual, ['Trust']].values[0].split('%')[0]) <= 70:
#                         current_shift_val = -1
# #                 if current_shift_val.empty:
# #                     print(csv_name)
                    
#                 temp_labels[j] = current_shift_val
    
#     temp_labels[np.isnan(temp_labels)] = -1.0
    
#     pdb_f_labeled = pdb_f.copy()
    
    
#     pdb_f_labeled['label'] = temp_labels
    
#     if np.sum(np.isnan(temp_labels)) > 0:
#         print('nan', csv_name)
#         a = temp_labels
#     current_out_path = os.path.join(out_labeled_pdb_dir, str(i) + '.csv')
    
#     pdb_f_labeled.to_csv(current_out_path, index = False)

100%|███████████████████████████████████████| 1999/1999 [07:33<00:00,  4.41it/s]


In [20]:
np.isnan(csv_f.loc[corresponding_residual, ['Trust']].values[0])

True

In [ ]:
not np.isnan(csv_f.loc[corresponding_residual, ['Trust']].values[0])

In [ ]:
corresponding_residual, [current_atom_type]

In [ ]:
np.unique(pdb_f['Residual_num']), len(csv_f)

In [ ]:
csv_f['Residue']

In [ ]:
pdb_f_labeled

In [ ]:
4 % 2

In [ ]:
0.5 ** (0.5)